In [2]:
from datasets import load_dataset

dataset = load_dataset("argilla/ultrafeedback-binarized-preferences-cleaned")

In [3]:
dataset['train']

Dataset({
    features: ['source', 'prompt', 'chosen', 'chosen-rating', 'chosen-model', 'rejected', 'rejected-rating', 'rejected-model'],
    num_rows: 60917
})

In [ ]:
from collections import Counter
Counter([len(x['chosen']) for x in dataset['train']])

In [2]:
import json
new_data = []
for i, item in enumerate(dataset['train']):
    assert item['chosen'][0]['content'] == item['rejected'][0]['content'] == item['prompt']
    chosen_response = item['chosen'][1]['content']
    chosen_model = item['chosen-model']
    chosen_rating = item['chosen-rating']
    rejected_response = item['rejected'][1]['content']
    rejected_model = item['rejected-model']
    rejected_rating = item['rejected-rating']
    instruction = item['prompt']
    assert chosen_rating > rejected_rating, "Chosen response should be better than rejected response"
    new_data.append({
        "id": "ultrafeedback_binarized_preferences_cleaned_" + str(i),
        "instruction": instruction,
        "input": "",
        "candidates": [
            {
                "text": chosen_response,
                "model": chosen_model,
                "decoding_method": "unknown",
                "scores": {
                    "human_preference": chosen_rating,
                }
            },
            {
                "text": rejected_response,
                "model": rejected_model,
                "decoding_method": "unknown",
                "scores": {
                    "human_preference": rejected_rating,
                }
                
            }
        ]
    })

import random
random.seed(42)
random_idx = random.sample(range(len(new_data)), 1000)
train_data = [new_data[i] for i in range(len(new_data)) if i not in random_idx]
val_data = [new_data[i] for i in range(len(new_data)) if i in random_idx]
with open("train_data_prepared.json", "w") as f:
    json.dump(train_data, f, indent=4)

with open("val_data_prepared.json", "w") as f:
    json.dump(val_data, f, indent=4)

    